In [16]:
import numpy as np
import cv2
import pandas as pd
import os
import matplotlib.pyplot as plt
from lu_vp_detect import VPDetection
import math
from tqdm import tqdm
import itertools

In [17]:
def collinearity_error(a,b,c):
    ba = (a[0] - b[0], a[1] - b[1])
    bc = (c[0] - b[0], c[1] - b[1])

    # Dot product and magnitudes
    dot_product = ba[0]*bc[0] + ba[1]*bc[1]
    mag_ba = math.sqrt(ba[0]**2 + ba[1]**2)
    mag_bc = math.sqrt(bc[0]**2 + bc[1]**2)

    # Angle in radians
    angle_rad = math.acos(dot_product / (mag_ba * mag_bc))

    # Convert to degrees
    return math.degrees(angle_rad)

def to_hom(p):
    return np.array([p[0], p[1], 1.0], dtype=float)

def line_from_points(p1, p2):
    return np.cross(to_hom(p1), to_hom(p2))

def intersection(l1, l2):
    p = np.cross(l1, l2)
    return p / p[2]

def calc_third_point(a, vp1, c, vp2):
    lA_vp = line_from_points(a, vp1)
    lC_vp = line_from_points(c, vp2)
    D_h = intersection(lA_vp, lC_vp)
    return D_h[:2]

def distance(p1, p2):
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

def distances_from_vp(a, b, c, d, vp1, vp2):
    lad = line_from_points(a, d)
    lcb = line_from_points(c, b)
    lab = line_from_points(a, b)
    ldc = line_from_points(d, c)
    int1 = intersection(lad, lcb)
    int2 = intersection(lab, ldc)
    dist1 = distance(int1, vp1)
    dist2 = distance(int2, vp2)
    return np.sum([dist1, dist2])


def find_missing_corner(A, B, C, vp1, vp2, vp3):
    """
    Given 3 image points (A, B, C) known to be adjacent corners of a square
    after perspective projection, compute the 4th corner.
    """
    width = 1024
    height = 768
    # calculate all candidate D points for all combination of vanishing points
    D_cand = []
    combinations = []
    for p in itertools.permutations([vp1, vp2, vp3], 2):
        D_cand.append(calc_third_point(A, p[0], C, p[1]))
        combinations.append(p)
    del_idx = []
    for i, D_h in enumerate(D_cand):
        #plt.scatter(D_h[0], D_h[1], c='blue')
        if D_h[0] < 0 or D_h[0] > width or D_h[1] < 0 or D_h[1] > height:
            del_idx.append(i) 
        # remove if D_h int not the lowest point among A, B, C
        elif not (D_h[1] >= A[1] and D_h[1] >= B[1] and D_h[1] >= C[1]):
            del_idx.append(i)
            
            
    #print("Candidate D points:", D_cand)
    for i in sorted(del_idx, reverse=True):
        del D_cand[i]
        del combinations[i]
    #print("Candidate D after removal:", D_cand)
    if len(D_cand) == 0:
        return None 
    errors = []
    for i, D, P in zip(range(len(D_cand)), D_cand, combinations):
        err = distances_from_vp(A, B, C, D, P[0], P[1])
        errors.append(err)
        #print(f"Combination {i}: D {D} VP1 {P[0]}, VP2 {P[1]}, Total Error: {err}")
    min_err_idx = np.argmin(errors)
    D_h = D_cand[min_err_idx]
    return D_h[:2]



In [18]:
def get_vp(image):
    #gray = cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)
    gray = cv2.threshold(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), 1, 255, cv2.THRESH_BINARY)[1]
    vp = vp_detector.find_vps(gray)
    #plt.imshow(vp_detector.create_debug_VP_image())
    return vp

In [19]:
def process_images(image):
    alpha_channel = image[:,:,-1]
    ret, thresh = cv2.threshold(alpha_channel, 1, 255, cv2.THRESH_BINARY)
    #plt.imshow(image, cmap='gray')
    contours, hierarchies = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    # find top left an and rightmost points
    points = [] 
    for cnt in contours:
        for point in cnt:
            points.append(point[0])
    points = np.array(points)
    leftmost = points[np.argmin(points[:, 0])]
    rightmost = points[np.argmax(points[:, 0])]
    topmost = points[np.argmin(points[:, 1])]
    #plt.imshow(image)
    #plt.scatter(leftmost[0], leftmost[1], color='red', marker='x')
    #plt.scatter(topmost[0], topmost[1], color='green', marker='x')
    #plt.scatter(rightmost[0], rightmost[1], color='blue', marker='x')
    return leftmost, topmost, rightmost
    

In [20]:
lsd = cv2.createLineSegmentDetector()

focal_pixel = (1024 * 0.5) / np.tan(39.6 * 0.5 * np.pi/180)
vp_detector = VPDetection(length_thresh=15, focal_length=focal_pixel)
img_dir = 'E:/dev/pygo_synthetic/renders_new'
annotations_file = 'labels.parquet.gz'
labels =  pd.read_parquet(os.path.join(img_dir, annotations_file))
corners_list = []
for index, row in tqdm(labels.iterrows()):
    
    img = cv2.imread(os.path.join(img_dir, row['filename']))
    left, top, right = process_images(img.copy())
    vp = get_vp(img.copy())
    w, h = img.shape[1], img.shape[0]
    vp1 = vp_detector.vps_2D[0]
    vp2 = vp_detector.vps_2D[1]
    vp3 = vp_detector.vps_2D[2]
    #print(f"VP1: {vp1}, VP2: {vp2}, VP3: {vp3}")

    #print(bottom)
    #plt.imshow(img)
    bottom = find_missing_corner(left, top, right, vp1, vp2, vp3)
    if bottom is None:
        print(f"Could not find bottom corner for image {row['filename']}")
        corners_list.append(np.array([[0,0],[0,0],[0,0],[0,0]]))
        continue
    #plt.scatter(bottom[0], bottom[1], color='yellow')
    #plt.show()
    w, h = img.shape[1], img.shape[0]
    # Normalize coordinates
    left_norm = (left[0]/w, left[1]/h)
    top_norm = (top[0]/w, top[1]/h)
    right_norm = (right[0]/w, right[1]/h)
    bottom_norm = (bottom[0]/w, bottom[1]/h)
    corners = np.asarray([left_norm, top_norm, right_norm, bottom_norm])
    corners_list.append(corners)
labels['corneres'] = corners_list
#labels.to_parquet(os.path.join(img_dir, 'labels_with_corners.parquet.gz'))
    
    

9it [00:00, 11.09it/s]

Could not find bottom corner for image go_19_rim_only_solid_229_234_244_rnd24r49_Wood016_2K-JPG_0.2_0.001_onyx_ivory_007.png


13it [00:01, 11.49it/s]

Could not find bottom corner for image go_19_studio_left_solid_244_244_244_rnd16r62_Wood020_2K-JPG_0.151_0.001_onyx_white_010.png
Could not find bottom corner for image go_19_studio_left_solid_244_244_244_rnd58r68_Wood022_2K-JPG_0.035_0.001_black_ivory_011.png


23it [00:02, 11.54it/s]

...

Could not find bottom corner for image go_19_studio_right_solid_244_244_244_rnd73r36_Wood034_2K-JPG_0.005_0.001_black_ivory_1585.png


1597it [02:31, 10.06it/s]

Could not find bottom corner for image go_19_studio_left_solid_229_234_244_rnd79r33_Wood009_2K-JPG_0.005_0.001_onyx_ivory_1594.png
Could not find bottom corner for image go_19_studio_right_solid_15_15_15_rnd71r37_Wood017_2K-JPG_0.035_0.0015_onyx_ivory_pattern_1595.png


1605it [02:31, 10.56it/s]


KeyboardInterrupt: 